In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor,IsolationForest
from sklearn.preprocessing import StandardScaler,KBinsDiscretizer,LabelEncoder
from sklearn.metrics import mean_squared_error,f1_score
from sklearn.kernel_approximation import Nystroem
from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import StratifiedKFold

from catboost import Pool, cv,CatBoostClassifier,CatBoostRegressor

from tqdm import tqdm

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
train_df = pd.read_csv('train_df.csv')
test_df = pd.read_csv('test_df.csv')
submission_df = pd.read_csv('sample_submission.csv')

data = train_df[train_df['label'] == 1].dropna()

# Isolation forests for anomaly detection on noise columns

In [ ]:
def anomaly_pred(col, train_df=train_df, test_df=test_df, folds=3):
    labels = train_df['label'].values
    X = train_df[col].values

    X_train_df = train_df[col].values
    X_test_df = test_df[col].values
    
    skf = StratifiedKFold(n_splits=3)

    validation_scores = []
    models = []

    train_preds = np.zeros(train_df.shape[0])
    test_preds = np.zeros(test_df.shape[0])

    for train_index, test_index in skf.split(X, labels):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        X_train = X_train.reshape((-1,1))
        X_test = X_test.reshape((-1,1))

        model = IsolationForest(random_state=0).fit(X_train)
        preds = model.predict(X_test).clip(0,1).reshape(y_test.shape)
        validation_score = f1_score(y_test, preds)

        train_preds += model.predict(X_train_df.reshape(-1,1)).reshape(X_train_df.shape).clip(0,1)
        test_preds += model.predict(X_test_df.reshape(-1,1)).reshape(X_test_df.shape).clip(0,1)

    #     print('Validation score:' , validation_score)

        validation_scores.append(validation_score)
        models.append(model)
        
    train_df[f'{col}_anomaly'] = np.where(train_preds > 2, 1, 0)
    test_df[f'{col}_anomaly'] = np.where(test_preds > 2, 1, 0)
    return validation_scores

In [ ]:
cols = ['fare','additional_fare','duration','meter_waiting','meter_waiting_fare','meter_waiting_till_pickup']
for col in tqdm(cols):
    validation_scores = anomaly_pred(col)
    print(f'col:{col}, mean:{np.mean(validation_scores):.4f}, std:{np.std(validation_scores):.4f}')

# Multi column anomaly detection

In [ ]:
def anomaly_pred_multi(cols, train_df=train_df, test_df=test_df, folds=3):
    labels = train_df['label'].values
    X = train_df[cols].values

    X_train_df = train_df[cols].values
    X_test_df = test_df[cols].values
    
    skf = StratifiedKFold(n_splits=3)

    validation_scores = []
    models = []

    train_preds = np.zeros(train_df.shape[0])
    test_preds = np.zeros(test_df.shape[0])
#     print(X.shape)

    for train_index, test_index in skf.split(X, labels):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = labels[train_index], labels[test_index]

        model = IsolationForest(random_state=0).fit(X_train)
        preds = model.predict(X_test).clip(0,1)
        validation_score = f1_score(y_test, preds)

        train_preds += model.predict(X_train_df).clip(0,1)
        test_preds += model.predict(X_test_df).clip(0,1)

    #     print('Validation score:' , validation_score)

        validation_scores.append(validation_score)
        models.append(model)
    name = '_'.join(cols)
    train_df[f'{name}_anomaly'] = np.where(train_preds > 2, 1, 0)
    test_df[f'{name}_anomaly'] = np.where(test_preds > 2, 1, 0)
    return validation_scores

In [ ]:
cols = ['fare','additional_fare','duration','meter_waiting','meter_waiting_fare','meter_waiting_till_pickup']
for i, col_1 in enumerate(cols):
    for col_2 in cols[i+1:]:
        validation_scores = anomaly_pred_multi([col_1,col_2])
        print(f'cols:{[col_1,col_2]}, mean:{np.mean(validation_scores):.4f}, std:{np.std(validation_scores):.4f}')

In [ ]:
for i, col_1 in enumerate(cols):
    for col_2 in cols[i+1:]:
        j = cols.index(col_2)
        for col_3 in cols[j+1:]:
            validation_scores = anomaly_pred_multi([col_1,col_2,col_3])
            print(f'cols:{[col_1,col_2,col_3]}, mean:{np.mean(validation_scores):.4f}, std:{np.std(validation_scores):.4f}')

In [ ]:
for i, col_1 in enumerate(cols):
    for col_2 in cols[i+1:]:
        j = cols.index(col_2)
        for col_3 in cols[j+1:]:
            k = cols.index(col_3)
            for col_4 in cols[k+1:]:                
                validation_scores = anomaly_pred_multi([col_1,col_2,col_3,col_4])
                print(f'cols:{[col_1,col_2,col_3,col_4]}, mean:{np.mean(validation_scores):.4f}, std:{np.std(validation_scores):.4f}')

In [ ]:
validation_scores = anomaly_pred_multi(cols)
np.mean(validation_scores) , np.std(validation_scores)

# Classifier

In [ ]:
params = {
    'loss_function':'Logloss',
    'random_state':0,
    'early_stopping_rounds':50,
    'eval_metric':'F1',
#     'class_weights':class_weights
}

In [ ]:
train_df.columns

In [ ]:
features = [
    'fare_anomaly',
    'additional_fare_anomaly', 
    'duration_anomaly',
    'meter_waiting_anomaly', 
    'meter_waiting_fare_anomaly',
    'meter_waiting_till_pickup_anomaly', 
    'additional_fare_duration_anomaly',
    'additional_fare_meter_waiting_anomaly',
    'additional_fare_meter_waiting_fare_anomaly',
    'additional_fare_meter_waiting_till_pickup_anomaly',
    'duration_meter_waiting_anomaly', 
    'duration_meter_waiting_fare_anomaly',
    'duration_meter_waiting_till_pickup_anomaly',
    'meter_waiting_meter_waiting_fare_anomaly',
    'meter_waiting_meter_waiting_till_pickup_anomaly',
    'meter_waiting_fare_meter_waiting_till_pickup_anomaly',
    'additional_fare_duration_meter_waiting_anomaly',
    'additional_fare_duration_meter_waiting_fare_anomaly',
    'additional_fare_duration_meter_waiting_till_pickup_anomaly',
    'additional_fare_meter_waiting_meter_waiting_fare_anomaly',
    'additional_fare_meter_waiting_meter_waiting_till_pickup_anomaly',
    'additional_fare_meter_waiting_fare_meter_waiting_till_pickup_anomaly',
    'duration_meter_waiting_meter_waiting_fare_anomaly',
    'duration_meter_waiting_meter_waiting_till_pickup_anomaly',
    'duration_meter_waiting_fare_meter_waiting_till_pickup_anomaly',
    'meter_waiting_meter_waiting_fare_meter_waiting_till_pickup_anomaly',
    'additional_fare_duration_meter_waiting_meter_waiting_fare_anomaly',
    'additional_fare_duration_meter_waiting_meter_waiting_till_pickup_anomaly',
    'additional_fare_duration_meter_waiting_fare_meter_waiting_till_pickup_anomaly',
    'additional_fare_meter_waiting_meter_waiting_fare_meter_waiting_till_pickup_anomaly',
    'duration_meter_waiting_meter_waiting_fare_meter_waiting_till_pickup_anomaly',
    
]

cat_features = [
    'additional_fare_anomaly', 
    'duration_anomaly',
    'meter_waiting_anomaly', 
    'meter_waiting_fare_anomaly',
    'meter_waiting_till_pickup_anomaly', 
    'additional_fare_duration_anomaly',
    'additional_fare_meter_waiting_anomaly',
    'additional_fare_meter_waiting_fare_anomaly',
    'additional_fare_meter_waiting_till_pickup_anomaly',
    'duration_meter_waiting_anomaly', 
    'duration_meter_waiting_fare_anomaly',
    'duration_meter_waiting_till_pickup_anomaly',
    'meter_waiting_meter_waiting_fare_anomaly',
    'meter_waiting_meter_waiting_till_pickup_anomaly',
    'meter_waiting_fare_meter_waiting_till_pickup_anomaly',
    'additional_fare_duration_meter_waiting_anomaly',
    'additional_fare_duration_meter_waiting_fare_anomaly',
    'additional_fare_duration_meter_waiting_till_pickup_anomaly',
    'additional_fare_meter_waiting_meter_waiting_fare_anomaly',
    'additional_fare_meter_waiting_meter_waiting_till_pickup_anomaly',
    'additional_fare_meter_waiting_fare_meter_waiting_till_pickup_anomaly',
    'duration_meter_waiting_meter_waiting_fare_anomaly',
    'duration_meter_waiting_meter_waiting_till_pickup_anomaly',
    'duration_meter_waiting_fare_meter_waiting_till_pickup_anomaly',
    'meter_waiting_meter_waiting_fare_meter_waiting_till_pickup_anomaly',
    'additional_fare_duration_meter_waiting_meter_waiting_fare_anomaly',
    'additional_fare_duration_meter_waiting_meter_waiting_till_pickup_anomaly',
    'additional_fare_duration_meter_waiting_fare_meter_waiting_till_pickup_anomaly',
    'additional_fare_meter_waiting_meter_waiting_fare_meter_waiting_till_pickup_anomaly',
    'duration_meter_waiting_meter_waiting_fare_meter_waiting_till_pickup_anomaly',

]

In [ ]:
labels = train_df['label'].values
train_df = train_df.drop(['label'], axis=1)[features]

In [ ]:
submission_pool = Pool(data=test_df[features], cat_features=cat_features)
train_df_pool = Pool(data=train_df[features], cat_features=cat_features)

In [ ]:
skf = StratifiedKFold(n_splits=3)
validation_scores = []
submission_preds = np.zeros(submission_df.shape[0])
train_preds = np.zeros(train_df.shape[0])
test_preds = np.zeros(test_df.shape[0])
train_pools = []
models = []
for train_index, test_index in skf.split(train_df, labels):
    X_train, X_test = train_df.iloc[train_index,:], train_df.iloc[test_index,:]
    y_train, y_test = labels[train_index], labels[test_index]
    train_pool = Pool(data=X_train, label=y_train,cat_features=cat_features)
    test_pool = Pool(data=X_test, label=y_test, cat_features=cat_features)    
    model = CatBoostClassifier(**params)
    model.fit(X=train_pool, eval_set=test_pool,verbose=10)
    pred = model.predict(test_pool)
    validation_score = model.best_score_['validation']['F1']
    print('Validation f1',validation_score)
    validation_scores.append(validation_score)
    models.append(model)
    train_pools.append(train_pool)
    submission_preds += model.predict(submission_pool)
    train_preds += model.predict_proba(train_df_pool)[:,1]
    test_preds += model.predict_proba(submission_pool)[:,1]

In [ ]:
np.mean(validation_scores), np.std(validation_scores)

In [ ]:
submission_df['prediction'] = np.where(submission_preds > 2, 1, 0)
submission_df.to_csv('submission_anomaly.csv',index=False)

In [ ]:
train_df.to_csv('train_df_anomaly.csv',index=False)
test_df.to_csv('test_df_anomaly.csv',index=False)

In [ ]:
stacking_train_df = pd.read_csv('stacking_train_df.csv')
stacking_test_df = pd.read_csv('stacking_test_df.csv')

stacking_train_df['catboost_anomaly'] = train_preds
stacking_test_df['catboost_anomaly'] = submission_preds

In [ ]:
stacking_test_df.to_csv('stacking_test_df.csv',index=False)
stacking_train_df.to_csv('stacking_train_df.csv',index=False)